# 机器阅读理解任务

## Step1 加载数据集

In [45]:
# 从本地加载数据集
from datasets import DatasetDict
datasets = DatasetDict.load_from_disk("D:/data/mrc")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [1]:
from datasets import load_dataset, DatasetDict

datasets = load_dataset("cmrc2018", cache_dir="D:\data\mrc")

datasets

d:\Anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since cmrc2018 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at D:\data\mrc\cmrc2018\default\0.0.0\137f2c45a24275fb68f6961c4d357f46288886aa (last modified on Sat Aug 24 16:40:46 2024).


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [2]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## Step2 数据预处理

In [3]:
from transformers import AutoTokenizer, DefaultDataCollator

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

tokenizer

d:\Anaconda3\envs\transformers\lib\site-packages\transformers\utils\hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [19]:
sample_datasets = datasets['train'].select(range(10))
sample_datasets
print(sample_datasets[0])

{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。', 'question': '范廷颂是什么时候被任为主教的？', 'answers': {'text': ['1963年'], 'answer_start': [30]}}


In [20]:
tokenizer_example = tokenizer(text=sample_datasets['question'], 
                             text_pair=sample_datasets['context'], 
                             max_length=384, 
                             truncation="only_second",
                             padding='max_length', 
                             return_offsets_mapping=True)
#print(tokenizer_example['input_ids'][0], len(tokenizer_example['input_ids'][0]))
print(tokenizer_example.keys())
print(tokenizer_example['offset_mapping'][0]) # offset_mapping对应的是input_ids中的每个token在原始文本中的起始和结束位置

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])
[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (9

In [8]:
print(list(zip(tokenizer_example['input_ids'][0], tokenizer_example['token_type_ids'][0])))

[(101, 0), (5745, 0), (2455, 0), (7563, 0), (3221, 0), (784, 0), (720, 0), (3198, 0), (952, 0), (6158, 0), (818, 0), (711, 0), (712, 0), (3136, 0), (4638, 0), (8043, 0), (102, 0), (5745, 1), (2455, 1), (7563, 1), (3364, 1), (3322, 1), (8020, 1), (8024, 1), (8021, 1), (8024, 1), (1760, 1), (1399, 1), (924, 1), (4882, 1), (185, 1), (5735, 1), (4449, 1), (8020, 1), (8021, 1), (8024, 1), (3221, 1), (6632, 1), (1298, 1), (5384, 1), (7716, 1), (1921, 1), (712, 1), (3136, 1), (3364, 1), (3322, 1), (511, 1), (9155, 1), (2399, 1), (6158, 1), (818, 1), (711, 1), (712, 1), (3136, 1), (8039, 1), (8431, 1), (2399, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (1921, 1), (712, 1), (3136, 1), (3777, 1), (1079, 1), (2600, 1), (3136, 1), (1277, 1), (2134, 1), (2429, 1), (5392, 1), (4415, 1), (8039, 1), (8447, 1), (2399, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (2600, 1), (712, 1), (3136, 1), (8024, 1), (1398, 1), (2399, 1), (2399, 1), (2419, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (3364, 

In [21]:
offset_mapping = tokenizer_example.pop('offset_mapping')
print(offset_mapping)
#offset_mapping = tokenizer_example['offset_mapping']
#print(offset_mapping)

[[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), 

In [26]:
print(tokenizer_example.sequence_ids(2))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [30]:
for idx, offset in enumerate(offset_mapping):
    answer = sample_datasets[idx]['answers']
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])

    # 定位答案在token中的起始位置和结束位置
    # 一种策略： 拿到context的起始位置和结束位置，然后从两端向答案进行逼近，context的sequence_ids的起始位置是1，结束位置是最后一个None - 1
    context_start = tokenizer_example.sequence_ids(idx).index(1)
    context_end = tokenizer_example.sequence_ids(idx).index(None, context_start) - 1
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -= 1
        end_token_pos = token_id

    #print(idx, answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    #print("token answer decode:", tokenizer.decode(tokenizer_example['input_ids'][idx][start_token_pos:end_token_pos+1]))

0 {'text': ['1963年'], 'answer_start': [30]} 30 35 17 382 47 48
token answer decode: 1963 年
1 {'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
2 {'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382 100 124
token answer decode: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
3 {'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 382 0 0
token answer decode: [CLS]
4 {'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 382 0 0
token answer decode: [CLS]
5 {'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 382 47 62
token answer decode: 《 全 美 超 级 模 特 儿 新 秀 大 赛 》 第 十 季
6 {'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 382 232 238
token answer decode: 有 前 途 的 新 面 孔
7 {'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 382 0 0
token answer decode: [CLS]
8 {'text': ['售货员'], 'ans

In [34]:
def process_data(examples):
    tokenizer_example = tokenizer(text=examples['question'], 
                             text_pair=examples['context'], 
                             max_length=384, 
                             truncation="only_second",
                             padding='max_length', 
                             return_offsets_mapping=True)
    
    offset_mapping = tokenizer_example.pop('offset_mapping')

    start_positions = []
    end_positions = []

    for idx, offset in enumerate(offset_mapping):
        answer = examples['answers'][idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # 定位答案在token中的起始位置和结束位置
        # 一种策略： 拿到context的起始位置和结束位置，然后从两端向答案进行逼近，context的sequence_ids的起始位置是1，结束位置是最后一个None - 1
        context_start = tokenizer_example.sequence_ids(idx).index(1)
        context_end = tokenizer_example.sequence_ids(idx).index(None, context_start) - 1
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -= 1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    tokenizer_example['start_positions'] = start_positions
    tokenizer_example['end_positions'] = end_positions
    return tokenizer_example

In [35]:
tokenied_data = datasets.map(process_data, batched=True, remove_columns=datasets["train"].column_names)
tokenied_data

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map: 100%|██████████| 1002/1002 [00:00<00:00, 1848.26 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## Step3: 创建模型

In [37]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step4: TrainingArguments

In [39]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 总的训练轮数
    per_device_train_batch_size=32,  # 每个GPU的训练批次大小
    per_device_eval_batch_size=32,   # 每个GPU的评估批次大小
    eval_strategy="epoch",         # 每个epoch进行一次评估
    save_strategy="epoch",           # 每个epoch进行一次保存
    logging_steps=50,              # 每多少步进行一次日志记录
)

## Step5: Trainer

In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,                         # 训练的模型
    args=args,                           # 训练参数
    train_dataset=tokenied_data["train"], # 训练数据集
    eval_dataset=tokenied_data["validation"], # 评估数据集
    data_collator=DefaultDataCollator()
)

## Step6: 训练模型

In [41]:
trainer.train()

  0%|          | 0/951 [00:00<?, ?it/s]

KeyboardInterrupt: 